In [1]:
import numpy as np
from transitions import Machine

STEP_SIZE = 10  # simulation/calculation step size, make sure smaller than any relevant values
    # use the step size of the DAC setting 0.15V - 1.65V with ~200 codes.

# Note: cooling is positive in GHz and heating is negative in GHz
# Laser parameters
NUM_OF_LASERS = 8   # number of lasers and/or MRMs
LASER_FREQ_SPACING = 400  # in GHz
LASER_FREQ_UNCERTAINTY = 50  # ± this number in GHz
# MRM parameters
MRM_FSR = 1800  # in GHz
MRM_FREQ_SPACING = -200  # in GHz, negative freq is longer wavelength
MRM_FREQ_UNCERTAINTY = 200  # in GHz
# CRR parameters
CRR_FSR = 1800  # in GHz
CRR_FREQ_SPACING = -200  # in GHz, negative freq is longer wavelength
CRR_FREQ_UNCERTAINTY = 100  # in GHz

# will track the resonances -5*FSR over, ..., 0*FSR over, ... 4*FSR over
# for both MRMs and CRRs.
WHICH_RESONANCES = list(range(-5, 5, 1))
BARREL_ROLL_SEQ = [0, 5, 1, 6, 2, 7, 3, 4]


class Lasers(object):

    """Lasers
    Initialize the lasers. Flags:
    - variations (bool): whether to enable laser wavelength uncertainty
    - corners (bool): if True, only evaluate the corners of the wavelength
        uncertainties. Won't do anything if `variations` is `False`.
    - reverse (bool): if True, go in decreasing frequency.
        (higher frequnency first in the list).
    - offest (int): specify the offset from 0. Otherwise, the lowest frequency
        will always be 0 GHz.

    The object has two important properties:
    - `laser_lines` (list): gives us the frequency of the lasers.
    - `brightness` (list): will say `bright` if that laser is uncaptured.
        Capture a laser line with the function `capture_laser_line`.
        That specific laser brightness will change to the MRM/CRR ring idx.
    """

    def __init__(self,
                 variations=False,
                 corners=False,
                 reverse=False,
                 offset=0) -> None:
        self._delta = None

        # start by having all the laser lines 'bright'
        # will be replaced by the MRM number that captures that laser
        # Note: need to considerq upstream vs. downstream
        self._brightness = ['bright'] * NUM_OF_LASERS

        # If reverse, the first laser line will have the highest frequency
        self._reverse = reverse

        # offset specifies the additive frequency offset from 0
        self._offset = offset

        self.populate_laser_lines(variations, corners)

    def populate_laser_lines(self, variations, corners):
        # the first laser line is always at 0 GHz
        laser_lines = np.arange(
            NUM_OF_LASERS, dtype=int) * LASER_FREQ_SPACING

        if self._reverse:
            laser_lines = np.flip(laser_lines)

        laser_lines += self._offset

        if variations:
            if corners:  # only evaluate the corners
                delta = np.random.randint(2, size=laser_lines.size)
                delta = 2*delta - 1  # convert to [-1, +1] random numbers
                delta *= LASER_FREQ_UNCERTAINTY
            else:
                delta = np.random.uniform(-1*LASER_FREQ_UNCERTAINTY,
                                          LASER_FREQ_UNCERTAINTY,
                                          laser_lines.size)
            # round to the nearest STEP_SIZE
            delta = np.round(delta/STEP_SIZE, decimals=0) * STEP_SIZE
            delta = delta.astype(int)

            self._delta = delta
            laser_lines += delta

        self._laser_lines = laser_lines

    def capture_laser_line(self, which_laser, which_ring, type='mrm'):
        if type == 'mrm':
            assert self._brightness[which_laser] == 'bright', \
                f"Laser line {which_laser} (at {self._laser_lines[which_laser]} GHz) should be bright to be captured"
        else:  # 'crr'
            assert self._brightness[which_laser] == which_ring, \
                f"Laser line {which_laser} (at {self._laser_lines[which_laser]} GHz) should be numbered {which_ring} to be captured"
        # which_ring will modulate this laser
        self._brightness[which_laser] = which_ring

    @property
    def laser_lines(self):
        return self._laser_lines

    @property
    def brightness(self):
        return self._brightness

    def __str__(self) -> str:
        return f"Lines: {self.laser_lines}, Brightness: {self.brightness}"

    def __repr__(self) -> str:
        return self.__str__()


class RingFSM(object):
    """RingFSM
    Base class for all rings: MRMs and CRRs.
    However, they will be modified below, use MRM_FSR and CRR_FSR instead.
    """

    states = ['idle', 'cold', 'search', 'lock']
    INITIAL_COOLDOWN = 200  # in GHz
    # Using this default value because it should clear laser uncertainty and
    # the ring resonance uncertainty at ± 100 GHz.
    DEFAULT_FSR = 1800  # in GHz

    def __init__(self,
                 resonances,
                 lasers,
                 initial_loc=0,
                 this_ring=0,  # this ring's index in the bank
                 previous_ring=None,  # None if there's no previous ring, otherwise supply the ring idx
                 cool_down_by=INITIAL_COOLDOWN,  # cool down by this amt in GHz
                 step_size=STEP_SIZE,
                 heating_threshold=-1*DEFAULT_FSR,
                 cooling_threshold=INITIAL_COOLDOWN,
                 ring_fsr=DEFAULT_FSR,
                 ) -> None:
        self.laser_locs_seen = []
        self.laser_brightness_seen = []
        self.laser_indices_seen = []
        self.loc = initial_loc  # location of the resonance

        self.locked_laser_idx = None

        self._initial_loc = initial_loc
        self._this_ring = this_ring
        self._previous_ring = previous_ring
        self._initial_resonances = resonances
        self._lasers = lasers

        self._cool_down_by = cool_down_by
        self._step_size = step_size
        self._ring_fsr = abs(ring_fsr)

        assert heating_threshold >= -1*self._ring_fsr and heating_threshold <= 0
        self._heating_threshold = heating_threshold
        assert cooling_threshold <= self._ring_fsr and cooling_threshold >= 0
        self._cooling_threshold = cooling_threshold

        self._found_all_lines = False

        self.machine = Machine(
            model=self, states=RingFSM.states, initial='idle')

        # 'start' starts the search algorithm
        self.machine.add_transition(trigger='start',
                                    source='idle',
                                    dest='cold',
                                    after='cool_step')

        # once the ring has cooled, start searching
        self.machine.add_transition(trigger='keep_searching',
                                    source='cold',
                                    dest='search',
                                    after='heat_step')

        # 'reset' puts the ring back to idle from any state
        self.machine.add_transition(trigger='reset',
                                    source='*',
                                    dest='idle',
                                    after='reset_resonances')

        # will 'keep_searching' until we find all the laser lines
        self.machine.add_transition(
            trigger='keep_searching',
            source='search',
            dest='search',
            after='heat_step')

        # lock ring once the laser lines have been found
        self.machine.add_transition(
            trigger='lock_to_line',
            source='search',
            dest='lock',
            after='lock_to_laser',
            conditions=['found_all_lines'])

    def reset_resonances(self):
        # reset everything
        self.loc = self._initial_loc
        self.laser_locs_seen = []
        self.laser_brightness_seen = []
        self.laser_indices_seen = []
        self._found_all_lines = False

    def cool(self, amount):
        self.loc += amount

    def heat(self, amount):
        self.loc -= amount

    def cool_step(self):
        # cool the resonances by the cool down by amount
        self.cool(self._cool_down_by)

    def heat_step(self):
        # check if there's a laser signal to be found
        found = self._check_laser_signal()
        # found is tuple (bool, idx | None)
        # if found: (True, laser idx), and if not found: (False, None)
        if found[0]:
            self.laser_locs_seen.append(self.loc)
            self.laser_brightness_seen.append(
                self._lasers.brightness[found[1]])
            self.laser_indices_seen.append(found[1])

        # check if we have found all lasers
        self._check_exit_search_condition()

        # move the resonance location by specified step_size
        self.heat(self._step_size)

    def lock_to_laser(self):

        # Correct the range of numbers to between [-1*FSR, cooling_threshold]
        candidate_loc = self.laser_locs_seen[-1]
        # bring any number beyond 0 and -1*fsr to this range
        candidate_loc = candidate_loc % (-1*self._ring_fsr)
        # bring any number between -1*fsr and the cooling limit to a positive value (cold)
        cooling_limit = self._cooling_threshold - self._ring_fsr
        if candidate_loc > (-1*self._ring_fsr) and candidate_loc <= cooling_limit:
            candidate_loc += self._ring_fsr

        # jump to the laser signal we want to capture
        self.loc = candidate_loc
        # check and capture that laser signal
        self._check_laser_signal(capture=True)

    def _capture_line(self, idx, ring_number):
        return self._lasers.capture_laser_line(idx, ring_number, type='mrm')

    def _check_laser_signal(self, capture=False):
        # check if there's any laser signal at this location
        # and, also decide to capture.
        found_laser = False
        idx = None
        for r in self.resonances:
            if (r in self._lasers.laser_lines):
                # get the index
                idx = np.where(self._lasers.laser_lines == r)
                assert len(idx) == 1
                idx = idx[0][0]
                if capture:
                    self._capture_line(idx, self._this_ring)
                    self.locked_laser_idx = idx
                found_laser = True
        # if there's a laser there, found_laser will return that laser's index
        return (found_laser, idx)

    def _check_exit_search_condition(self):
        # This is the crux of the search algorithm.
        # In this case, we consider the simple algo.
        # if we have seen one dim and one bright, we have found all lines
        if self._previous_ring is None:  # the first ring in the bank
            if len(self.laser_locs_seen) >= 2:
                self._found_all_lines = True
        else:  # all the other rings
            if len(self.laser_locs_seen) >= 2:
                # NB: this line is the only significant change in the simplified algorithm
                if self.laser_brightness_seen[-2] != 'bright' and self.laser_brightness_seen[-1] == 'bright':
                    self._found_all_lines = True

    @property
    def found_all_lines(self):
        return self._found_all_lines

    @property
    def resonances(self):
        return self.loc + self._initial_resonances

    @property
    def within_threshold(self):
        # if the number is beyond the cooling and heating threshold, then it's outside of threshold
        # otherwise, it's within threshold
        return self.loc >= self._heating_threshold and self.loc <= self._cooling_threshold


class MRM_FSM(RingFSM):

    INITIAL_COOLDOWN = 200  # in GHz

    def __init__(self,
                 resonances,  # a list of this ring's resonances
                 lasers,  # laser object
                 initial_loc=0,  # initial heating/cooling in GHz
                 this_ring=0,  # this ring's index in the bank
                 previous_ring=None,  # None if there's no previous ring, otherwise supply the ring idx
                 cool_down_by=INITIAL_COOLDOWN,  # cool down by this amt in GHz before searching
                 step_size=STEP_SIZE,
                 heating_threshold=-1*MRM_FSR,
                 cooling_threshold=INITIAL_COOLDOWN
                 ) -> None:

        super().__init__(resonances, lasers, initial_loc, this_ring,
                         previous_ring, cool_down_by, step_size, heating_threshold, cooling_threshold, MRM_FSR)


class CRR_FSM(RingFSM):

    INITIAL_COOLDOWN = 200  # in GHz

    def __init__(self,
                 resonances,  # a list of this ring's resonances
                 lasers,  # laser object
                 initial_loc=0,  # initial heating/cooling in GHz
                 this_ring=0,  # this ring's index in the bank
                 previous_ring=None,  # None if there's no previous ring, otherwise supply the ring idx
                 cool_down_by=INITIAL_COOLDOWN,  # cool down by this amt in GHz before searching
                 step_size=STEP_SIZE,
                 heating_threshold=-1*CRR_FSR,
                 cooling_threshold=INITIAL_COOLDOWN
                 ) -> None:

        super().__init__(resonances, lasers, initial_loc, this_ring,
                         previous_ring, cool_down_by, step_size, heating_threshold, cooling_threshold, CRR_FSR)

    def _check_exit_search_condition(self):
        # This is the crux of the search algorithm.
        if len(self.laser_locs_seen) >= 1:
            # CRR needs to capture laser used by an MRM of the same idx
            if self.laser_brightness_seen[-1] == self._this_ring:
                self._found_all_lines = True

    def _capture_line(self, idx, ring_number):
        return self._lasers.capture_laser_line(idx, ring_number, type='crr')

In [2]:
from collections import deque
import copy


def populate_resonances(freq_spacing,
                        freq_uncertainty,
                        fsr,
                        global_variations=False,
                        variations=False,
                        corners=False):
    """populate_resonances (base function)
    USE THE FUNCTIONS `populate_mrm_resonances` and `populate_crr_resonances`
    BELOW INSTEAD.

    Initialize the resonances of the rings. Flags:
    - freq_spacing (int): frequency spacing from one ring to the other.
    - freq_uncertainty (int): local resonance frequency uncertainty will be
        sampled from a uniform dist between [-1*freq_uncertainty, freq_uncertainty].
    - fsr (int): the FSR of each ring.
    - global_variations (bool): whether to enable global resonance frequency
        uncertainty with a uniform distribution of [-1*MRM_FSR, MRM_FSR]
    - variations (bool): whether to enable local resonance frequency uncertainty
    - corners (bool): if True, only evaluate the corners of the frequency
        uncertainties. Won't do anything if `variations` is `False`.

    Returns:
    - lines (np.ndarray): A numpy 2D-array, organized where the first dim
        corresponds to which ring in the array and the second dim
        corresponds to which resonance frequency of that ring.

        For example:
        lines[3] will return the resonances of the 4th ring in the bank.
        lines[3][0] will return the first resonance (lowest value in freq)
            of the 4th ring in the bank.
    """

    lines = np.arange(NUM_OF_LASERS, dtype=np.int64) * freq_spacing
    lines = np.vstack([lines] * len(WHICH_RESONANCES)).T

    fsr_shifts = np.array(WHICH_RESONANCES) * fsr
    lines += fsr_shifts

    if variations:  # assume we keep the FSR to be the same
        if corners:  # only evaluate the corners
            delta = np.random.randint(2, size=NUM_OF_LASERS)
            delta = 2*delta - 1  # convert to [-1, +1] random numbers
            delta *= freq_uncertainty
        else:
            delta = np.random.uniform(-1*freq_uncertainty,
                                      freq_uncertainty,
                                      NUM_OF_LASERS)
        # round to the nearest STEP_SIZE
        delta = np.round(delta/STEP_SIZE, decimals=0) * STEP_SIZE

        delta = delta.astype(int)
        delta = delta[:, np.newaxis]
        lines += delta

    if global_variations:  # the global variation can be as much as ±1 FSR
        global_delta = np.random.uniform(-1*fsr, fsr)
        global_delta = np.round(global_delta/STEP_SIZE, decimals=0) * STEP_SIZE
        global_delta = global_delta.astype(int)
        lines += global_delta

    return lines


def populate_mrm_resonances(global_variations=False,
                            variations=False,
                            corners=False):
    return populate_resonances(MRM_FREQ_SPACING, MRM_FREQ_UNCERTAINTY, MRM_FSR,
                               global_variations, variations, corners)


def populate_crr_resonances(global_variations=False,
                            variations=False,
                            corners=False):
    return populate_resonances(CRR_FREQ_SPACING, CRR_FREQ_UNCERTAINTY, CRR_FSR,
                               global_variations, variations, corners)


# HELPER FUNCTIONS
def check_barrel_roll(list1, list2):
    # check for barrel roll in sequence or reverse sequence
    assert len(list1) == len(list2)

    def check_equivalent(list1, list2):
        d1 = deque(list1)
        d2 = deque(list2)

        equivalent = d1 == d2

        for _ in range(len(list1)):
            d1.rotate(1)
            if d1 == d2:
                equivalent = True

        return equivalent

    list1_ = copy.deepcopy(list1)
    equivalent = check_equivalent(list1_, list2)
    list1_.reverse()
    equivalent = equivalent or check_equivalent(list1_, list2)
    return equivalent


def calculate_maxima(amt_heat_applied):
    colder = [n for n in amt_heat_applied if n >= 0]
    hotter = [n for n in amt_heat_applied if n <= 0]

    max_cool = max(colder) if len(colder) > 0 else 0
    max_heat = min(hotter) if len(hotter) > 0 else 0
    max_diff = max_cool - max_heat
    return max_cool, max_heat, max_diff

In [3]:
# Try out the algorithm with a single try
lasers = Lasers(reverse=True, offset=-4*LASER_FREQ_SPACING, variations=True, corners=False)
all_mrm_resonances_ = populate_mrm_resonances(variations=False, corners=False)
all_mrm_resonances = all_mrm_resonances_
all_mrm_resonances[2] = all_mrm_resonances_[6]
all_mrm_resonances[6] = all_mrm_resonances_[2]

all_mrms = []

# Lock all the MRMs
for idx in range(NUM_OF_LASERS):
    previous_ring = None if idx == 0 else (idx-1)

    mrm = MRM_FSM(resonances=all_mrm_resonances[idx],
                  lasers=lasers,
                  this_ring=idx,
                  previous_ring=previous_ring)
    mrm.start()

    while not mrm.found_all_lines:
        mrm.keep_searching()

    mrm.lock_to_line()
    all_mrms.append(mrm)

    print(f"Locked to laser # {mrm.locked_laser_idx}")
    print(lasers.brightness)

all_crr_resonances = populate_crr_resonances(variations=True, corners=False)
all_crrs = []

# Now lock all the CRRs
for idx in range(NUM_OF_LASERS):
    import copy
    lasers_ = copy.deepcopy(lasers)
    previous_ring = None if idx == 0 else (idx-1)
    crr = CRR_FSM(resonances=all_crr_resonances[idx],
                  lasers=lasers_,
                  this_ring=idx,
                  previous_ring=previous_ring)
    crr.start()

    while not crr.found_all_lines:
        crr.keep_searching()

    crr.lock_to_line()
    all_crrs.append(crr)


mrm_lasers_locked_to = [int(m.locked_laser_idx) for m in all_mrms]
crr_lasers_locked_to = [int(m.locked_laser_idx) for m in all_crrs]

# check for barrel roll in sequence or reverse sequence
print("Barrel roll check?", check_barrel_roll(mrm_lasers_locked_to, BARREL_ROLL_SEQ))

print(f"MRM and CRR lasers: {mrm_lasers_locked_to} & {crr_lasers_locked_to}")

amt_heat_applied = [m.loc for m in all_mrms]
print(f"Amount of heat applied for MRM: {amt_heat_applied}")
amt_heat_applied = [m.loc for m in all_crrs]
print(f"Amount of heat applied for CRR: {amt_heat_applied}")

Locked to laser # 4
['bright', 'bright', 'bright', 'bright', 0, 'bright', 'bright', 'bright']
Locked to laser # 0
[1, 'bright', 'bright', 'bright', 0, 'bright', 'bright', 'bright']
Locked to laser # 5
[1, 'bright', 'bright', 'bright', 0, 2, 'bright', 'bright']
Locked to laser # 1
[1, 3, 'bright', 'bright', 0, 2, 'bright', 'bright']
Locked to laser # 6
[1, 3, 'bright', 'bright', 0, 2, 4, 'bright']
Locked to laser # 2
[1, 3, 5, 'bright', 0, 2, 4, 'bright']
Locked to laser # 7
[1, 3, 5, 'bright', 0, 2, 4, 6]
Locked to laser # 3
[1, 3, 5, 7, 0, 2, 4, 6]
Barrel roll check? True
MRM and CRR lasers: [4, 0, 5, 1, 6, 2, 7, 3] & [4, 0, 5, 1, 6, 2, 7, 3]
Amount of heat applied for MRM: [-400, -440, -1360, -430, -440, -400, -390, -400]
Amount of heat applied for CRR: [-420, -390, -350, -410, -500, -340, -480, -340]


In [4]:
# Run Monte Carlo and get distribution

import pandas as pd
import copy
from tqdm.notebook import tqdm

eval_corners = False
global_variations = True  # main difficulty
local_variations = True
laser_variations = True

cycling_threshold_mrm = 4/9  # ratio of FSR --> 800 GHz
cycling_threshold_crr = 6/9  # ratio of FSR
cooling_threshold_mrm = 2/9  # ratio of FSR --> 400 GHz
cooling_threshold_crr = 2/9  # ratio of FSR

""" Note this setting will always work
cycling_threshold_mrm = 2/9  # ratio of FSR
cycling_threshold_crr = 7/9  # ratio of FSR
cooling_threshold_mrm = 2/9  # ratio of FSR
cooling_threshold_crr = 2/9  # ratio of FSR
"""

# With cycling:
cycling_freqs = [-k * 100 for k in range(18)]
# Without cycling: uncomment the one below for no cycling
# cycling_freqs = [0]

num_trials = 100
trial_results_mrm = []
trial_results_crr = []

for _ in tqdm(range(num_trials)):
    lasers = Lasers(reverse=True, offset=-4*LASER_FREQ_SPACING,
                    variations=laser_variations, corners=eval_corners)
    all_mrm_resonances = populate_mrm_resonances(global_variations=global_variations,
                                                 variations=local_variations,
                                                 corners=eval_corners)
    all_crr_resonances = populate_crr_resonances(global_variations=global_variations,
                                                 variations=local_variations,
                                                 corners=eval_corners)

    for ctry, cfreq in enumerate(cycling_freqs):
        # Get all the MRMs to lock properly first
        lasers_ = copy.deepcopy(lasers)
        all_mrms = []
        for idx in range(NUM_OF_LASERS):
            previous_ring = None if idx == 0 else (idx-1)
            mrm = MRM_FSM(resonances=all_mrm_resonances[idx],
                            lasers=lasers_,
                            heating_threshold=-cycling_threshold_mrm * MRM_FSR,
                            cooling_threshold=cooling_threshold_mrm * MRM_FSR,
                            cool_down_by=cooling_threshold_mrm * MRM_FSR,
                            initial_loc=cfreq,
                            this_ring=idx,
                            previous_ring=previous_ring)
                            
            # Run the search algorithm for each MRM
            mrm.start()
            while not mrm.found_all_lines:
                mrm.keep_searching()
            mrm.lock_to_line()
            all_mrms.append(mrm)

        # Save all the MRM information here
        mrm_laser_sequence = [int(m.locked_laser_idx) for m in all_mrms]
        amt_heat_applied_mrm = [m.loc for m in all_mrms]
        max_cool_mrm, max_heat_mrm, max_diff_mrm = \
            calculate_maxima(amt_heat_applied_mrm)

        # Lock all the CRRs to the corresponding MRMs
        all_crrs = []
        for idx in range(NUM_OF_LASERS):
            lasers__ = copy.deepcopy(lasers_)
            previous_ring = None if idx == 0 else (idx-1)
            crr = CRR_FSM(resonances=all_crr_resonances[idx],
                            lasers=lasers__,
                            this_ring=idx,
                            previous_ring=idx-1,
                            heating_threshold=-cycling_threshold_crr * MRM_FSR,
                            cooling_threshold=cooling_threshold_crr * MRM_FSR,
                            cool_down_by=cooling_threshold_crr * MRM_FSR)
            crr.start()
            while not crr.found_all_lines:
                crr.keep_searching()
            crr.lock_to_line()
            all_crrs.append(crr)

        # Save all the CRR information here
        crr_laser_sequence = [int(m.locked_laser_idx) for m in all_crrs]
        amt_heat_applied_crr = [m.loc for m in all_crrs]
        max_cool_crr, max_heat_crr, max_diff_crr = calculate_maxima(
            amt_heat_applied_crr)

        # Check if cycling is needed
        mrm_cycling_condition = abs(max_heat_mrm) > (MRM_FSR * cycling_threshold_mrm)
        crr_cycling_condition = abs(max_heat_crr) > (MRM_FSR * cycling_threshold_crr)
        cycling_condition = mrm_cycling_condition or crr_cycling_condition
        # if `cycling_condition == True`: cycle, otherwise break the loop
        if not cycling_condition:
            break

    mrm_result = {'laser_sequence': mrm_laser_sequence,
                  # check if barrel roll equivalent
                  'barrel_roll': check_barrel_roll(mrm_laser_sequence, BARREL_ROLL_SEQ),
                  'max_cool': max_cool_mrm,
                  'max_heat': max_heat_mrm,
                  'max_diff': max_diff_mrm}

    mrm_numbers = list(range(NUM_OF_LASERS))
    # (-) value is heat and (+) value is cool
    mrm_heat_result = dict(zip(mrm_numbers, amt_heat_applied_mrm))
    mrm_result.update(mrm_heat_result)

    # check if we were able to lock within the threshold
    mrm_threshold = [m.within_threshold for m in all_mrms]
    crr_threshold = [m.within_threshold for m in all_crrs]
    successful_lock = mrm_threshold + crr_threshold
    successful_lock = bool(np.array(successful_lock).all())

    overall_result = {'cycling_trial': ctry, 'successful_lock': successful_lock}
    mrm_result.update(overall_result)

    crr_result = {'laser_sequence': crr_laser_sequence,
                  'mrm_crr_equivalent': mrm_laser_sequence == crr_laser_sequence,
                  'max_cool': max_cool_crr,
                  'max_heat': max_heat_crr,
                  'max_diff': max_diff_crr}

    crr_numbers = list(range(NUM_OF_LASERS))
    # (-) value is heat and (+) value is cool
    heat_result = dict(zip(crr_numbers, amt_heat_applied_crr))
    crr_result.update(heat_result)

    trial_results_mrm.append(mrm_result)
    trial_results_crr.append(crr_result)

trial_results_mrm = pd.DataFrame(trial_results_mrm)
trial_results_crr = pd.DataFrame(trial_results_crr)

  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
# Now plot the results of the Monte Carlo runs
import plotly.express as px

# Plot MRM
# Heat and cool amount
fig = px.histogram(np.array(trial_results_mrm[range(NUM_OF_LASERS)]).reshape(-1))  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = STEP_SIZE)
fig.update_layout(xaxis_title="Heat (-) & Cool (+) amount (GHz)", yaxis_title="Count", title="MRM heating & cooling", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(-MRM_FSR, MRM_FSR+tick_size, tick_size))
    )
)
fig.show()
fig.write_image(f"mrm_heating_{MRM_FREQ_UNCERTAINTY}GHz.png", height=450, width=1089)

# Plot the maximum differences
fig = px.histogram(trial_results_mrm, x="max_diff")  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = STEP_SIZE)
fig.update_layout(xaxis_title="Heat amount difference between rings (GHz)", yaxis_title="Count", title="MRM max diff between rings", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0, 2*MRM_FSR+tick_size, tick_size))
    )
)
fig.show()
fig.write_image(f"mrm_diff_{MRM_FREQ_UNCERTAINTY}GHz.png", height=450, width=1089)

# Plot CRR
# Heat and cool amount
fig = px.histogram(np.array(trial_results_crr[range(NUM_OF_LASERS)]).reshape(-1))  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = STEP_SIZE)
fig.update_layout(xaxis_title="Heat (-) & Cool (+) amount (GHz)", yaxis_title="Count", title="CRR heating & cooling", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(-CRR_FSR, CRR_FSR+tick_size, tick_size))
    )
)
fig.show()
fig.write_image(f"crr_heating_{CRR_FREQ_UNCERTAINTY}GHz.png", height=450, width=1089)

# Plot the maximum differences
fig = px.histogram(trial_results_crr, x="max_diff")  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = STEP_SIZE)
fig.update_layout(xaxis_title="Heat amount difference between rings (GHz)", yaxis_title="Count", title="CRR max diff between rings", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0, 2*CRR_FSR+tick_size, tick_size))
    )
)
fig.show()
fig.write_image(f"crr_diff_{CRR_FREQ_UNCERTAINTY}GHz.png", height=450, width=1089)

In [6]:
# check the results: barrel-roll and meeting our threshold requirements
print("All in barrel-roll sequence? ", trial_results_mrm['barrel_roll'].all())
print("All locked within the heating limit?", trial_results_mrm['successful_lock'].all())

# display all the trial results in a markdown table
from IPython.display import display, Markdown
display(Markdown("# MRM results"))
display(Markdown(trial_results_mrm.to_markdown()))

display(Markdown("# CRR results"))
display(Markdown(trial_results_crr.to_markdown()))

All in barrel-roll sequence?  True
All locked within the heating limit? False


# MRM results

|    | laser_sequence           | barrel_roll   |   max_cool |   max_heat |   max_diff |    0 |    1 |    2 |     3 |    4 |    5 |    6 |    7 |   cycling_trial | successful_lock   |
|---:|:-------------------------|:--------------|-----------:|-----------:|-----------:|-----:|-----:|-----:|------:|-----:|-----:|-----:|-----:|----------------:|:------------------|
|  0 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        260 |          0 |        260 |  130 |  260 |  180 |   150 |  210 |  140 |   50 |   10 |               0 | True              |
|  1 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -610 |        610 | -110 | -290 | -220 |  -320 | -100 | -390 | -330 | -610 |               1 | True              |
|  2 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        160 |       -410 |        570 |  160 |  130 | -130 |   -70 | -120 | -410 | -150 | -110 |               0 | True              |
|  3 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         20 |       -320 |        340 | -150 | -150 |  -30 |    20 | -110 | -130 | -260 | -320 |               0 | True              |
|  4 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -760 |        760 | -360 | -530 | -590 |  -540 | -500 | -570 | -330 | -760 |               2 | True              |
|  5 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        230 |       -230 |        460 |  210 |  -20 | -140 |  -120 |  -90 |  230 | -140 | -230 |               0 | True              |
|  6 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        150 |       -210 |        360 |  -60 |   40 |  150 |   120 |  -50 | -200 |  130 | -210 |               1 | True              |
|  7 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |          0 |       -480 |        480 |  -10 |  -90 | -210 |  -480 | -370 | -290 | -130 | -200 |               0 | True              |
|  8 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -720 |        720 | -540 | -210 | -490 |  -440 | -590 | -720 | -580 | -540 |               5 | True              |
|  9 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         70 |       -280 |        350 |  -40 |  -60 |  -10 |  -260 | -280 |  -40 |   70 |  -90 |               0 | True              |
| 10 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        220 |        -50 |        270 |  120 |   70 |  -20 |   -50 |  220 |   40 |   20 |  -40 |               0 | True              |
| 11 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        240 |        -80 |        320 |  140 |  210 |  240 |   210 |  -80 |   30 |  160 |   40 |               0 | True              |
| 12 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -770 |        770 | -570 | -630 | -340 |  -590 | -400 | -770 | -550 | -580 |               7 | True              |
| 13 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        120 |       -240 |        360 |  120 | -180 | -150 |  -220 | -100 | -240 | -220 |  -40 |               0 | True              |
| 14 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        180 |       -240 |        420 |  180 | -220 |  180 |   -60 | -190 | -240 |  -30 | -180 |               0 | True              |
| 15 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        350 |       -200 |        550 |   10 |  -10 |  -40 |   180 |  350 |   10 |  210 | -200 |               0 | True              |
| 16 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         90 |       -140 |        230 |   70 |   90 | -140 |    20 | -140 |   50 |  -60 |  -30 |               0 | True              |
| 17 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        160 |       -200 |        360 |  160 |  -40 | -200 |  -180 |   60 |  100 |  110 |   10 |               0 | True              |
| 18 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        400 |          0 |        400 |  220 |  400 |   50 |   210 |   70 |   80 |   10 |   40 |               0 | True              |
| 19 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         30 |       -400 |        430 | -130 |  -90 |   20 |  -400 |   30 |   -0 | -240 |   10 |               1 | True              |
| 20 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        380 |       -150 |        530 |  160 |  380 |  250 |   350 |   -0 |   30 |   60 | -150 |               0 | True              |
| 21 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        300 |       -210 |        510 |   60 |  300 |   30 |    80 |  140 |  -50 |   10 | -210 |               0 | True              |
| 22 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        200 |       -270 |        470 |  200 |   90 | -120 |  -130 |  140 |  -70 | -270 |   -0 |              17 | True              |
| 23 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         90 |       -340 |        430 |   90 |   40 | -240 |  -150 | -120 |  -20 | -290 | -340 |               0 | True              |
| 24 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -430 |        430 | -190 |  -10 |  -50 |  -100 | -330 | -250 | -430 | -400 |               3 | True              |
| 25 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -360 |        360 | -280 | -360 | -140 |  -210 | -350 | -300 | -240 | -340 |               3 | True              |
| 26 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        400 |          0 |        400 |  180 |  280 |  330 |   270 |  240 |  290 |  400 |  110 |               0 | True              |
| 27 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        150 |       -200 |        350 |   20 |   40 |  110 |   150 |  -30 | -200 |  -70 |   90 |               1 | True              |
| 28 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        280 |       -220 |        500 |  280 |  230 |  -50 |   250 |  260 |  210 | -220 |  -80 |              16 | True              |
| 29 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -660 |        660 | -310 | -520 | -570 |  -510 | -540 | -480 | -290 | -660 |               1 | True              |
| 30 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        180 |        -90 |        270 |   70 |  130 |  180 |   -70 |  130 |   60 |  -90 |  140 |               0 | True              |
| 31 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        170 |       -310 |        480 |   60 |  170 | -130 |  -130 | -230 | -180 |  -60 | -310 |               0 | True              |
| 32 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -760 |        760 | -470 | -550 | -200 |  -340 | -250 | -690 | -760 | -710 |               5 | True              |
| 33 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         80 |       -300 |        380 | -120 | -100 | -220 |   -60 |  -90 |  -60 | -300 |   80 |               0 | True              |
| 34 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        150 |       -350 |        500 |  150 |   30 |  -70 |  -350 | -290 |  -30 | -270 | -320 |               0 | True              |
| 35 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        170 |       -420 |        590 |  170 |  140 |  -50 |  -240 | -420 | -330 | -420 | -300 |               0 | True              |
| 36 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         70 |       -500 |        570 |   70 | -280 |   40 |  -240 | -120 |   20 | -170 | -500 |               0 | True              |
| 37 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        350 |      -1370 |       1720 |  140 |  350 |   60 | -1370 |  130 |  230 |  340 |  220 |              17 | False             |
| 38 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        350 |       -140 |        490 |   10 |  -90 |  350 |  -140 |  -70 |  -30 | -130 |   60 |               1 | True              |
| 39 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -410 |        410 | -160 | -200 |  -90 |  -410 | -180 |  -50 | -290 | -220 |               0 | True              |
| 40 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        250 |       -380 |        630 |  150 |   50 |  -50 |   250 |   30 |  100 | -380 |   20 |               0 | True              |
| 41 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |       -310 |        490 |  180 |  -70 | -310 |  -210 |  -50 | -300 |  -70 | -300 |               0 | True              |
| 42 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        220 |       -160 |        380 |  210 |  220 | -130 |   -90 |  -30 |  -60 | -160 | -100 |               0 | True              |
| 43 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -710 |        710 | -670 | -560 | -490 |  -660 | -390 | -470 | -560 | -710 |               8 | True              |
| 44 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -550 |        550 | -500 | -480 | -550 |  -230 | -510 | -180 | -490 | -490 |               4 | True              |
| 45 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -620 |        620 | -160 | -380 | -530 |  -230 | -170 | -350 | -620 | -500 |               2 | True              |
| 46 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -770 |        770 | -380 | -550 | -440 |  -770 | -610 | -330 | -580 | -510 |               4 | True              |
| 47 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        100 |       -270 |        370 |  -30 | -250 |   80 |   -50 |  100 |  -90 | -110 | -270 |               0 | True              |
| 48 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        110 |       -360 |        470 |  110 |  -40 |   -0 |  -360 | -310 |  -60 | -330 | -300 |               0 | True              |
| 49 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        270 |       -180 |        450 |   50 |  -80 |   60 |   270 |  270 |   30 |  160 | -180 |               0 | True              |
| 50 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        140 |        -60 |        200 |  -40 |  -50 |  140 |   110 |   60 |  -60 |  -40 |  120 |               0 | True              |
| 51 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         80 |       -360 |        440 |   80 |   10 |   40 |   -70 | -360 | -340 | -190 | -320 |               0 | True              |
| 52 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |       -580 |        580 |  -50 | -100 | -440 |  -430 | -500 | -580 | -280 | -530 |               1 | True              |
| 53 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |       -700 |        700 | -210 | -250 | -170 |  -520 | -550 | -450 | -330 | -700 |               0 | True              |
| 54 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         20 |       -360 |        380 |   20 |  -90 | -100 |   -20 | -360 | -280 | -260 | -240 |               0 | True              |
| 55 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         30 |       -280 |        310 |  -70 |   30 | -270 |  -170 | -130 | -110 | -240 | -280 |               0 | True              |
| 56 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -440 |        440 | -140 | -290 | -440 |  -100 | -130 | -240 | -140 | -320 |               0 | True              |
| 57 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        270 |        -20 |        290 |  140 |  200 |   80 |   190 |  270 |   90 |  -10 |  -20 |               0 | True              |
| 58 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         80 |       -330 |        410 |   50 |   80 | -220 |  -320 | -240 |   40 | -270 | -330 |               0 | True              |
| 59 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        340 |        -40 |        380 |   40 |  130 |  340 |   210 |  260 |   -0 |   70 |  -40 |               0 | True              |
| 60 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        210 |       -330 |        540 |  210 |  -30 |   70 |  -190 |   -0 | -120 | -310 | -330 |               0 | True              |
| 61 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        190 |        -90 |        280 |  140 |  100 |   30 |   190 |  130 |  -70 |   50 |  -90 |              15 | True              |
| 62 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        280 |        -70 |        350 |  130 |  120 |  200 |   160 |   70 |  280 |  100 |  -70 |               0 | True              |
| 63 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        320 |       -200 |        520 |   90 |  240 |  180 |   180 |  320 |  300 |  -70 | -200 |               0 | True              |
| 64 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        250 |       -120 |        370 |  -10 |  250 |   50 |   200 | -120 |   50 |   -0 |   30 |               1 | True              |
| 65 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        190 |       -300 |        490 |   30 |  -40 | -100 |   190 | -150 |   50 |  -40 | -300 |               0 | True              |
| 66 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        140 |       -170 |        310 |  -90 |  120 | -170 |  -100 | -170 |  140 |   60 |   20 |               0 | True              |
| 67 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -770 |        770 | -600 | -470 | -270 |  -390 | -380 | -440 | -700 | -770 |               7 | True              |
| 68 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -560 |        560 | -450 | -200 | -300 |  -560 | -250 | -540 | -510 | -470 |               2 | True              |
| 69 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        110 |       -100 |        210 |   90 | -100 |   60 |   -50 |  -80 |  -20 |  -40 |  110 |               0 | True              |
| 70 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        240 |       -240 |        480 |   60 |  240 |  220 |   -60 |  -70 | -240 |  -10 | -170 |               1 | True              |
| 71 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        110 |       -300 |        410 |  -60 | -180 | -120 |  -120 |  110 | -100 |   -0 | -300 |               0 | True              |
| 72 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        160 |       -210 |        370 | -210 | -130 | -140 |    50 |   -0 |  160 |  -10 |  -70 |               0 | True              |
| 73 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        270 |       -130 |        400 |  -40 |   -0 |  -80 |   270 | -130 |  250 |   20 |   70 |               0 | True              |
| 74 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        290 |        -70 |        360 |  290 |  -30 |  160 |   110 |  110 |  130 |  -70 |   30 |              16 | True              |
| 75 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         20 |       -150 |        170 |  -80 |  -10 |  -80 |   -30 | -110 |  -20 | -150 |   20 |               0 | True              |
| 76 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        190 |       -190 |        380 |  130 |   50 |   50 |  -190 |  130 |  -20 |  -80 |  190 |               0 | True              |
| 77 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        230 |       -150 |        380 | -100 |   50 |  130 |   170 |   30 | -150 |  230 |   -0 |               0 | True              |
| 78 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |       -430 |        430 | -120 | -130 | -290 |  -320 | -180 | -430 | -430 | -130 |               0 | True              |
| 79 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -520 |        520 | -350 | -460 | -260 |  -220 | -380 | -460 | -340 | -520 |               5 | True              |
| 80 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        150 |       -100 |        250 |  150 |   20 |   90 |    40 |   -0 |   90 |  -80 | -100 |               0 | True              |
| 81 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |          0 |       -650 |        650 | -450 | -340 | -340 |  -330 | -440 | -650 | -370 | -460 |               5 | True              |
| 82 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        280 |        -90 |        370 |  -20 |   -0 |  100 |   -90 |  280 |   80 |  180 |  -90 |               0 | True              |
| 83 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        140 |       -190 |        330 |  -50 |   90 |   90 |    50 |   80 | -100 |  140 | -190 |               0 | True              |
| 84 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        230 |       -310 |        540 |  230 |  180 |   40 |   -90 |   60 |  210 | -310 | -160 |               0 | True              |
| 85 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         -0 |       -300 |        300 |   -0 | -210 |  -10 |  -300 | -260 |  -40 | -270 |  -80 |              16 | True              |
| 86 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        160 |       -180 |        340 |   80 |  160 |  -30 |   -50 |  -40 |   80 | -180 |  160 |               0 | True              |
| 87 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        370 |        -30 |        400 |   -0 |  100 |  280 |    -0 |  140 |  370 |  -30 |  150 |               0 | True              |
| 88 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        310 |        -80 |        390 |  160 |  190 |  180 |   190 |  -80 |   30 |   20 |  310 |               0 | True              |
| 89 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        370 |        -60 |        430 |  -10 |  370 |   50 |   110 |  320 |  240 |   20 |  -60 |               0 | True              |
| 90 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         10 |       -630 |        640 |   10 | -330 | -420 |  -630 | -380 | -550 | -250 | -250 |               0 | True              |
| 91 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        330 |        -20 |        350 |  190 |  330 |  220 |   -20 |  140 |   30 |   90 |   20 |               0 | True              |
| 92 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        340 |       -150 |        490 |  130 |  280 |  310 |   340 |   50 |  210 | -120 | -150 |               0 | True              |
| 93 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |       -400 |        400 |  -90 |  -60 |  -60 |  -290 | -260 |  -80 | -400 | -350 |               1 | True              |
| 94 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        190 |       -230 |        420 |  190 |  160 | -170 |    20 |  120 | -190 | -230 |  -50 |               0 | True              |
| 95 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        170 |       -200 |        370 |  170 |   40 | -120 |   120 |   10 |   40 |   40 | -200 |               0 | True              |
| 96 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        170 |       -320 |        490 |  170 |  -70 |  -70 |    60 |  -90 | -130 |  -20 | -320 |               0 | True              |
| 97 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        270 |       -170 |        440 |  240 |  -50 |  220 |   270 |  -30 |   80 |  -20 | -170 |              17 | True              |
| 98 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |       -660 |        660 | -350 | -560 | -620 |  -450 | -550 | -660 | -620 | -520 |               4 | True              |
| 99 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        180 |       -320 |        500 | -120 |  180 | -210 |  -130 |   10 |  -90 |  100 | -320 |               0 | True              |

# CRR results

|    | laser_sequence           | mrm_crr_equivalent   |   max_cool |   max_heat |   max_diff |     0 |     1 |     2 |     3 |     4 |     5 |     6 |     7 |
|---:|:-------------------------|:---------------------|-----------:|-----------:|-----------:|------:|------:|------:|------:|------:|------:|------:|------:|
|  0 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -530 |        530 |  -300 |  -300 |  -350 |  -290 |  -530 |  -530 |  -360 |  -520 |
|  1 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        370 |          0 |        370 |   270 |   370 |   210 |   260 |   280 |   100 |    50 |    30 |
|  2 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1150 |       1150 |  -740 |  -770 |  -820 | -1110 | -1150 | -1020 |  -980 | -1000 |
|  3 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |         20 |       -300 |        320 |  -120 |    20 |   -50 |   -50 |   -40 |  -120 |   -80 |  -300 |
|  4 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        240 |        -10 |        250 |    40 |   240 |   120 |    90 |    70 |    60 |    40 |   -10 |
|  5 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1160 |       1160 |  -890 |  -820 |  -920 |  -860 |  -750 |  -920 | -1160 | -1160 |
|  6 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        290 |       -100 |        390 |   220 |   260 |   240 |   290 |   140 |  -100 |   110 |    30 |
|  7 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        320 |        -10 |        330 |   320 |   310 |   110 |    40 |   -10 |    30 |    10 |   190 |
|  8 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        280 |        -90 |        370 |   280 |   120 |   160 |   -90 |    40 |   -30 |   -10 |   -10 |
|  9 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -710 |        710 |  -610 |  -620 |  -640 |  -550 |  -710 |  -680 |  -580 |  -650 |
| 10 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -790 |        790 |  -650 |  -510 |  -520 |  -560 |  -410 |  -380 |  -790 |  -670 |
| 11 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -940 |        940 |  -600 |  -730 |  -600 |  -690 |  -910 |  -850 |  -940 |  -880 |
| 12 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        270 |        -10 |        280 |   250 |   260 |   170 |   210 |   270 |   160 |   100 |   -10 |
| 13 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        320 |          0 |        320 |   320 |   180 |   230 |   250 |   250 |    80 |   320 |   160 |
| 14 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -690 |        690 |  -320 |  -430 |  -430 |  -340 |  -690 |  -520 |  -450 |  -510 |
| 15 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |      -1130 |       1130 | -1010 |  -890 | -1090 |  -910 |  -890 | -1070 |  -950 | -1130 |
| 16 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        280 |        -50 |        330 |   280 |   190 |    50 |   -20 |   -40 |   -50 |    -0 |   -40 |
| 17 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        180 |       -120 |        300 |   180 |   -40 |  -120 |    10 |    30 |    20 |    70 |   110 |
| 18 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -490 |        490 |  -230 |  -170 |  -130 |  -130 |  -200 |  -280 |  -160 |  -490 |
| 19 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -750 |        750 |  -470 |  -520 |  -750 |  -730 |  -600 |  -720 |  -690 |  -590 |
| 20 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -770 |        770 |  -620 |  -540 |  -490 |  -540 |  -570 |  -540 |  -770 |  -700 |
| 21 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -870 |        870 |  -690 |  -500 |  -670 |  -500 |  -590 |  -870 |  -790 |  -710 |
| 22 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1170 |       1170 |  -910 |  -920 |  -830 |  -940 | -1030 | -1080 | -1170 | -1040 |
| 23 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -610 |        610 |  -310 |  -450 |  -430 |  -370 |  -330 |  -300 |  -580 |  -610 |
| 24 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        230 |        -60 |        290 |   150 |   180 |   200 |   100 |   230 |   -30 |    80 |   -60 |
| 25 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        260 |          0 |        260 |   260 |   200 |   240 |    20 |    80 |   150 |    30 |    50 |
| 26 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -300 |        300 |   -90 |  -180 |  -140 |  -170 |  -230 |  -260 |  -250 |  -300 |
| 27 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        170 |       -140 |        310 |   170 |    10 |    10 |    30 |  -140 |   -90 |  -140 |  -120 |
| 28 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1170 |       1170 |  -740 |  -940 |  -840 |  -770 |  -870 |  -830 | -1000 | -1170 |
| 29 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        310 |        -80 |        390 |   150 |   160 |   160 |   220 |   180 |   160 |   310 |   -80 |
| 30 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |      -1180 |       1180 |  -870 | -1180 | -1150 | -1100 | -1050 | -1120 | -1070 | -1140 |
| 31 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |      -1100 |       1100 |  -770 |  -890 |  -890 | -1100 |  -980 |  -910 |  -990 | -1000 |
| 32 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        240 |       -120 |        360 |   160 |    40 |   130 |    20 |   240 |  -100 |  -120 |   -30 |
| 33 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -810 |        810 |  -810 |  -600 |  -750 |  -640 |  -660 |  -720 |  -630 |  -740 |
| 34 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        130 |       -140 |        270 |   130 |   120 |    20 |   -80 |  -140 |   -60 |   -50 |   -40 |
| 35 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -500 |        500 |  -150 |  -140 |  -310 |  -360 |  -500 |  -440 |  -430 |  -430 |
| 36 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -910 |        910 |  -560 |  -630 |  -700 |  -700 |  -670 |  -580 |  -910 |  -850 |
| 37 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -970 |        970 |  -960 |  -850 |  -950 |  -900 |  -780 |  -970 |  -820 |  -960 |
| 38 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |         20 |       -280 |        300 |    20 |  -190 |  -130 |  -230 |  -260 |  -260 |  -270 |  -280 |
| 39 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -430 |        430 |  -230 |  -370 |  -200 |  -400 |  -430 |  -210 |  -430 |  -370 |
| 40 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |       -580 |        580 |  -360 |  -210 |  -330 |  -210 |  -200 |  -380 |  -580 |  -560 |
| 41 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |         90 |       -290 |        380 |    90 |    60 |  -240 |  -250 |  -240 |  -140 |  -270 |  -290 |
| 42 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |        240 |        -10 |        250 |   230 |   240 |   130 |    70 |   -10 |    10 |   130 |   120 |
| 43 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        220 |       -190 |        410 |   160 |    60 |   220 |   160 |    70 |   170 |  -190 |  -170 |
| 44 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        250 |          0 |        250 |   120 |   170 |   150 |   190 |   180 |   130 |   110 |   250 |
| 45 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        270 |        -70 |        340 |   170 |   220 |   240 |   220 |   270 |   200 |   -70 |    10 |
| 46 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        390 |          0 |        390 |   310 |   290 |   390 |   130 |   130 |   180 |   130 |   140 |
| 47 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -360 |        360 |  -210 |  -230 |  -140 |  -150 |  -150 |  -200 |  -230 |  -360 |
| 48 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -540 |        540 |  -190 |  -220 |  -200 |  -400 |  -420 |  -540 |  -350 |  -470 |
| 49 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -650 |        650 |  -580 |  -620 |  -500 |  -600 |  -600 |  -650 |  -610 |  -620 |
| 50 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        230 |       -120 |        350 |    20 |   150 |   200 |   230 |   -10 |  -120 |   -30 |   -30 |
| 51 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -740 |        740 |  -590 |  -600 |  -500 |  -710 |  -640 |  -740 |  -720 |  -650 |
| 52 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        260 |        -30 |        290 |   260 |   150 |   190 |   110 |   -30 |   -10 |    -0 |   -30 |
| 53 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        310 |         -0 |        310 |   160 |   240 |   250 |   310 |   270 |   190 |   190 |    -0 |
| 54 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -450 |        450 |  -100 |  -210 |  -130 |  -260 |  -450 |  -350 |  -290 |  -360 |
| 55 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         80 |       -150 |        230 |   -50 |    80 |  -150 |    -0 |   -30 |   -80 |   -60 |  -130 |
| 56 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1010 |       1010 |  -920 |  -880 |  -780 |  -780 | -1010 |  -970 |  -830 |  -840 |
| 57 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -990 |        990 |  -720 |  -850 |  -990 |  -910 |  -930 |  -940 |  -890 |  -920 |
| 58 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |      -1120 |       1120 |  -800 |  -760 | -1000 | -1100 | -1020 | -1100 | -1120 |  -970 |
| 59 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |          0 |      -1110 |       1110 |  -730 |  -780 |  -860 |  -800 |  -710 |  -950 |  -980 | -1110 |
| 60 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |          0 |       -780 |        780 |  -330 |  -710 |  -630 |  -770 |  -730 |  -730 |  -780 |  -600 |
| 61 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -970 |        970 |  -850 |  -780 |  -850 |  -970 |  -900 |  -860 |  -920 |  -780 |
| 62 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        170 |        -60 |        230 |   170 |   -20 |   -30 |   -10 |   -40 |   -60 |   -20 |    50 |
| 63 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        120 |       -240 |        360 |    50 |    50 |   120 |   -70 |   -10 |    30 |  -120 |  -240 |
| 64 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        350 |        -30 |        380 |   270 |   320 |   350 |   170 |   -30 |    70 |   -30 |    20 |
| 65 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        140 |       -230 |        370 |    20 |   140 |    20 |    70 |   110 |   -40 |  -160 |  -230 |
| 66 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |         70 |       -100 |        170 |  -100 |   -10 |    70 |   -50 |  -100 |    -0 |   -90 |   -20 |
| 67 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        220 |       -230 |        450 |    50 |    90 |   220 |   160 |   180 |   160 |  -230 |   -30 |
| 68 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        130 |        -10 |        140 |    20 |    60 |    50 |   130 |   -10 |    30 |    90 |    90 |
| 69 | [3, 4, 0, 5, 1, 6, 2, 7] | True                 |        160 |       -200 |        360 |   160 |  -180 |   -80 |  -200 |   -40 |  -180 |    10 |   -20 |
| 70 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -740 |        740 |  -490 |  -380 |  -340 |  -350 |  -720 |  -660 |  -740 |  -720 |
| 71 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -900 |        900 |  -790 |  -730 |  -820 |  -760 |  -790 |  -850 |  -900 |  -890 |
| 72 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |        100 |        -60 |        160 |   -20 |    10 |    30 |    -0 |    -0 |   100 |    -0 |   -60 |
| 73 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -190 |        190 |  -100 |   -60 |  -190 |   -90 |  -120 |   -60 |   -50 |   -60 |
| 74 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |      -1120 |       1120 |  -690 |  -820 |  -850 |  -870 |  -740 | -1020 | -1120 |  -970 |
| 75 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -390 |        390 |  -240 |  -320 |  -160 |  -260 |  -310 |  -260 |  -320 |  -390 |
| 76 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |        200 |       -150 |        350 |   200 |   100 |   200 |    70 |    60 |   -20 |  -150 |    30 |
| 77 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -730 |        730 |  -510 |  -460 |  -450 |  -600 |  -500 |  -500 |  -460 |  -730 |
| 78 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |       -230 |        230 |  -210 |  -120 |  -160 |  -170 |  -230 |  -200 |  -230 |  -210 |
| 79 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        350 |        -10 |        360 |   350 |   350 |   280 |   120 |   250 |    -0 |   -10 |    70 |
| 80 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -880 |        880 |  -580 |  -570 |  -640 |  -640 |  -650 |  -620 |  -680 |  -880 |
| 81 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |        290 |        -80 |        370 |   220 |   290 |   280 |   260 |   -20 |   -20 |    60 |   -80 |
| 82 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -860 |        860 |  -580 |  -690 |  -760 |  -750 |  -550 |  -680 |  -550 |  -860 |
| 83 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -430 |        430 |  -190 |  -150 |  -120 |  -170 |  -140 |  -230 |  -160 |  -430 |
| 84 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        220 |        -40 |        260 |   170 |    90 |   150 |    80 |   220 |   200 |   -40 |    20 |
| 85 | [4, 0, 5, 1, 6, 2, 7, 3] | True                 |          0 |      -1070 |       1070 |  -930 | -1040 |  -920 |  -880 | -1070 |  -990 |  -850 |  -820 |
| 86 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -750 |        750 |  -410 |  -440 |  -390 |  -420 |  -530 |  -750 |  -650 |  -710 |
| 87 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         40 |       -240 |        280 |    40 |   -80 |   -80 |    30 |    20 |   -10 |  -140 |  -240 |
| 88 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -440 |        440 |  -200 |  -120 |  -190 |  -440 |  -330 |  -360 |  -420 |  -400 |
| 89 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -810 |        810 |  -660 |  -660 |  -570 |  -540 |  -490 |  -620 |  -540 |  -810 |
| 90 | [7, 3, 4, 0, 5, 1, 6, 2] | True                 |          0 |       -600 |        600 |  -290 |  -270 |  -570 |  -510 |  -540 |  -600 |  -460 |  -480 |
| 91 | [2, 7, 3, 4, 0, 5, 1, 6] | True                 |          0 |       -880 |        880 |  -480 |  -480 |  -530 |  -670 |  -840 |  -690 |  -880 |  -770 |
| 92 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |          0 |       -240 |        240 |   -50 |  -190 |   -20 |   -40 |  -180 |  -240 |  -190 |  -210 |
| 93 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |        250 |       -150 |        400 |    80 |   250 |   210 |   200 |   150 |   210 |  -150 |   -10 |
| 94 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        160 |       -180 |        340 |   160 |   100 |   110 |    70 |   -60 |    10 |   -80 |  -180 |
| 95 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |          0 |       -850 |        850 |  -480 |  -650 |  -560 |  -480 |  -600 |  -510 |  -630 |  -850 |
| 96 | [5, 1, 6, 2, 7, 3, 4, 0] | True                 |         60 |       -290 |        350 |   -50 |   -90 |    60 |   -80 |    -0 |   -90 |  -100 |  -290 |
| 97 | [6, 2, 7, 3, 4, 0, 5, 1] | True                 |          0 |       -940 |        940 |  -590 |  -800 |  -770 |  -620 |  -940 |  -930 |  -870 |  -830 |
| 98 | [1, 6, 2, 7, 3, 4, 0, 5] | True                 |        400 |          0 |        400 |   400 |   400 |   260 |   320 |   390 |   210 |   280 |    50 |
| 99 | [0, 5, 1, 6, 2, 7, 3, 4] | True                 |        210 |          0 |        210 |    20 |   130 |   100 |   210 |    30 |   210 |    70 |    10 |